In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def read_data(filename, usecols=[2, 5, 8, 19, 20], dtype={'Primary Type': str, 'Arrest': bool, 'Latitude' : float, 'Longitude' : float}, **kwargs):

    data = pd.read_csv(filename, usecols=usecols, dtype=dtype, quotechar='"', parse_dates=['Date'], infer_datetime_format=True, index_col='Date', **kwargs)
    return data

In [ ]:
nrows = 100000
filename = '/Users/Elaad/Documents/Fun/DatAnalysis/datasets/Crimes_-_2001_to_present.csv'
crimedata = read_data(filename)

# Crimes as a function of time and date

In [59]:
allcrime_hourcounts = crimedata.index.hour.value_counts()
homicide_hourcounts = crimedata[crimedata['Primary Type']=='HOMICIDE'].index.hour.value_counts()

allcrime_daycounts = crimedata.index.day.value_counts()
homicide_daycounts = crimedata[crimedata['Primary Type']=='HOMICIDE'].index.day.value_counts()

allcrime_monthcounts = crimedata.index.month.value_counts()
homicide_monthcounts = crimedata[crimedata['Primary Type']=='HOMICIDE'].index.month.value_counts()

allcrime_yearcounts = crimedata.index.year.value_counts()
homicide_yearcounts = crimedata[crimedata['Primary Type']=='HOMICIDE'].index.year.value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.bar(allcrime_hourcounts.index, allcrime_hourcounts.values, label='All Crimes')
ax.bar(homicide_hourcounts.index, homicide_hourcounts.values*1000, label='Homicide')
ax.set_xlabel('Hour of Day', fontsize=18)
ax.set_ylabel('Number of Crimes', fontsize=18)
plt.legend(loc='best')

fig, ax = plt.subplots(1, 1)
ax.bar(allcrime_daycounts.index, allcrime_daycounts.values, label='All Crimes')
ax.bar(homicide_daycounts.index, homicide_daycounts.values*1000, label='Homicide')
ax.set_xlabel('Day of Month', fontsize=18)
ax.set_ylabel('Number of Crimes', fontsize=18)
plt.legend(loc='best')

fig, ax = plt.subplots(1, 1)
ax.bar(allcrime_monthcounts.index, allcrime_monthcounts.values, label='All Crimes')
ax.bar(homicide_monthcounts.index, homicide_monthcounts.values*1000, label='Homicide')
ax.set_xlabel('Month of Year', fontsize=18)
ax.set_ylabel('Number of Crimes', fontsize=18)
plt.legend(loc='best')

fig, ax = plt.subplots(1, 1)
ax.bar(allcrime_yearcounts.index, allcrime_yearcounts.values, label='All Crimes')
ax.bar(homicide_yearcounts.index, homicide_yearcounts.values*1000, label='Homicide')
ax.set_xlabel('Year', fontsize=18)
ax.set_ylabel('Number of Crimes', fontsize=18)
plt.legend(loc='best')

30